<a href="https://colab.research.google.com/github/Iterates/IFT-4022-TP2/blob/part-3/transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tâche #3 : Classification d'incidents avec des modèles *Transformers*

Toujours avec la même tâche et les mêmes fichiers de textes, utiliser la librairie HuggingFace pour accomplir cette tâche. On demande plus spécifiquement d’utiliser le modèle bert-base-uncased et un autre modèle de votre choix.
Les consignes associées à cette tâche sont:
-	Nom du notebook : transformer.ipynb
-	Tokenisation : Celle fournie par les tokeniseurs accompagnant les modèles transformers.
-	Plongements de mots : Ceux du modèle transformer.
-	Normalisation : Lettre en minuscule pour Bert. Aucune contrainte pour le 2e modèle.
-	Choix du 2e transformer: Un modèle encodeur préentraîné pour l’anglais. Le modèle ne doit pas être une autre version de Bert et doit être significativement différent. Utilisez un 2 fichier pour ce modèle si nécessaire (une copie de celui-ci).
-	Analyse : Comparer les résultats obtenus avec les 2 modèles transformers. Présentez également une comparaison globale des résultats obtenus avec tous les modèles utilisés dans ce travail et ceux du travail précédent (TP #1).


Vous pouvez ajouter au *notebook* toutes les cellules dont vous avez besoin pour votre code, vos explications ou la présentation de vos résultats. Vous pouvez également ajouter des sous-sections (par ex. des sous-sections 1.1, 1.2 etc.) si cela améliore la lisibilité.

Notes :
- Évitez les bouts de code trop longs ou trop complexe. Par exemple, il est difficile de comprendre 4-5 boucles ou conditions imbriquées. Si c'est le cas, définissez des sous-fonctions pour refactoriser et simplifier votre code.
- Expliquez sommairement votre démarche.
- Expliquez les choix que vous faites au niveau de la programmation et des modèles (si non trivial).
- Analyser vos résultats. Indiquez ce que vous observez, si c'est bon ou non, si c'est surprenant, etc.
- Une analyse quantitative et qualitative d'erreurs est intéressante et permet de mieux comprendre le comportement d'un modèle.

## 1. Création du jeu de données (*dataset*)

In [ ]:
import sys
import importlib.util

__COLAB__ = "google.colab" in sys.modules
__DEBUG__ = True


if __COLAB__:
    !pip install -U transformers dataset evaluate accelerate

In [ ]:
import numpy as np
import json
from pathlib import Path
import functools
from typing import List, Dict, TypedDict
from transformers import AutoTokenizer
from torch.utils.data import Dataset
from datasets import load_dataset

data_directory = "./data"
PRETRAINED_MODEL_NAME = "bert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)


def preprocess(batch: Dict[str, List]):
    return tokenizer(batch["text"], truncation=True) | {
        "label": [int(label) for label in batch["label"]]
    }


dataset = load_dataset(
    data_directory,
    data_files={
        "train": "incidents_train.json",
        "validation": "incidents_dev.json",
        "test": "incidents_test.json",
    },
)

dataset = dataset.map(preprocess, batched=True)

In [ ]:
dataset["train"]["input_ids"][0]

#### Gestion de l'étiquetage

In [ ]:
labels = set(dataset["train"]["label"])
id2label = {label: str(label) for label in labels}
label2id = {str(label): label for label in labels}
num_labels = len(labels)

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

## 2. Création de modèle(s)

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    PRETRAINED_MODEL_NAME, num_labels=num_labels, id2label=id2label, label2id=label2id
)

In [ ]:
import evaluate

accuracy = evaluate.load("accuracy")


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)

    return accuracy.compute(predictions=predictions, references=labels)

## 3. Entraînement de modèle(s)

In [ ]:
training_args = TrainingArguments(
    output_dir=f"./model/{PRETRAINED_MODEL_NAME.lower()}",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=25,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

## 4. Évaluation et analyse de résultats